In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
data_dir = Path().cwd().parent / "data"

In [3]:
def file_load(file_path):
    temp_dict = {"speech_id":[], "speech":[]}
    with open(file_path, 'r', encoding="ansi") as file:
        for i, line in enumerate(file):
            if i == 0: continue
            ls = line.split('|')
            ls[1] = ls[1].replace("\n","")
            ls[1] = ls[1].replace("\t", "")
            temp_dict["speech_id"].append(int(ls[0]))
            temp_dict["speech"].append(ls[1])
    return pd.DataFrame.from_dict(temp_dict)


In [4]:
congresses = [102, 103, 104, 105, 106, 107, 108]

In [5]:
df_list = []
speakers_df_list = []

for congress in congresses: 
    speeches = file_load(data_dir / f"hein-daily/speeches_{congress}.txt")
    speakers = pd.read_csv(data_dir / f"hein-daily/{congress}_SpeakerMap.txt", delimiter="|", encoding="ANSI")
    
    merged = speeches.merge(speakers, how="inner", on="speech_id").dropna()
    
    house = merged[(merged["chamber"] == "H") & (merged["nonvoting"] == "voting")].copy()
    
    house["congress"] = str(congress)
    
    df_list.append(house)
    speakers_df_list.append(speakers)

In [6]:
final_df = pd.concat(df_list, ignore_index=True)

In [8]:
final_df.to_csv(data_dir / "house_102_to_108.txt", sep="\t", index=False)

In [ ]:
speakers_df = pd.concat(speakers_df_list, ignore_index=True)

speakers_df = speakers_df[speakers_df["chamber"] == "H"].reset_index(drop=True)

speakers_df = speakers_df[["speakerid", "lastname", "firstname", "state", "district"]]

speakers_df["speakerid"] = speakers_df["speakerid"].map(str)

speakers_df = speakers_df.drop_duplicates(ignore_index=True)

In [ ]:
speakers_df.to_csv(data_dir / "speakers_102_to_108.csv", index=False)